In [17]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from common.layers import *

In [16]:
input_dim=(1,28,28)
conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1}
conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1}
conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1}
conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1}
conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1}
conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1}
hidden_size=50
output_size=10

pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
weight_init_scales = np.sqrt(2.0 / pre_node_nums)

params = {}
pre_channel_num = input_dim[0]

for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
    params[f'W{idx + 1}'] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
    params[f'b{idx + 1}'] = np.zeros(conv_param['filter_num'])
    
    pre_channel_num = conv_param['filter_num']

params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
params['b7'] = np.zeros(hidden_size)
params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
params['b8'] = np.zeros(output_size)

(50,)

In [45]:
class DeepConvNet:
    def __init__(self, input_dim=(1,28,28),
                conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                hidden_size=50, output_size=10):
        
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)
        
        self.params = {}
        pre_channel_num = input_dim[0]

        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params[f'W{idx + 1}'] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params[f'b{idx + 1}'] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
            
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)
        
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param_1['stride'], conv_param_1['pad'])
        self.layers['ReLU1'] = Relu()

        self.layers['Conv2'] = Convolution(self.params['W2'], self.params['b2'], conv_param_2['stride'], conv_param_2['pad'])
        self.layers['ReLU2'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)

        self.layers['Conv3'] = Convolution(self.params['W3'], self.params['b3'], conv_param_3['stride'], conv_param_3['pad'])
        self.layers['ReLU3'] = Relu()

        self.layers['Conv4'] = Convolution(self.params['W4'], self.params['b4'], conv_param_4['stride'], conv_param_4['pad'])
        self.layers['ReLU4'] = Relu()
        self.layers['Pool2'] = Pooling(pool_h=2, pool_w=2, stride=2)

        self.layers['Conv5'] = Convolution(self.params['W5'], self.params['b5'], conv_param_5['stride'], conv_param_5['pad'])
        self.layers['ReLU5'] = Relu()

        self.layers['Conv6'] = Convolution(self.params['W6'], self.params['b6'], conv_param_6['stride'], conv_param_6['pad'])
        self.layers['ReLU6'] = Relu()
        self.layers['Pool3'] = Pooling(pool_h=2, pool_w=2, stride=2)

        self.layers['Affine1'] = Affine(self.params['W7'], self.params['b7'])
        self.layers['ReLU7'] = Relu()
        self.layers['Dropout1'] = Dropout(0.5)

        self.layers['Affine2'] = Affine(self.params['W8'], self.params['b8'])
        self.layers['Dropout2'] = Dropout(0.5)

        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads[f'W{i+1}'] = self.layers[layer_idx].dW
            grads[f'W{i+1}'] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params[f'W{i+1}']
            self.layers[layer_idx].b = self.params[f'b{i+1}']


In [19]:
from dataset.mnist import load_mnist
from common.trainer import Trainer

In [46]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

In [47]:
network = DeepConvNet()
trainer = Trainer(network, x_train, t_train, x_test, t_test, epochs=20, mini_batch_size=100, optimizer='Adam',
       optimizer_param={'lr': 0.0001}, evaluate_sample_num_per_epoch=1000)
trainer.train()

AttributeError: 'str' object has no attribute 'forward'

In [48]:
class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]


In [49]:
network = DeepConvNet()
trainer = Trainer(network, x_train, t_train, x_test, t_test, epochs=20, mini_batch_size=100, optimizer='Adam',
       optimizer_param={'lr': 0.0001}, evaluate_sample_num_per_epoch=1000)
trainer.train()

train loss:2.3258490843582127
=== epoch:1, train acc:0.125, test acc:0.107 ===
train loss:2.3629406959846584
train loss:2.3374859452857395
train loss:2.3155852706013778
train loss:2.3462588073021964
train loss:2.407027948468714
train loss:2.328936252674367
train loss:2.30588806175774
train loss:2.292637120926865
train loss:2.3083755345057106
train loss:2.3118817728025713
train loss:2.300031909914821
train loss:2.2935708182599623
train loss:2.2890050200543643
train loss:2.3098324863711746
train loss:2.2869268308818578
train loss:2.307569936058799
train loss:2.282829031765878
train loss:2.288528149844966
train loss:2.2930295019159606
train loss:2.2838499628013387
train loss:2.2813892154717506
train loss:2.2738533294281025
train loss:2.284146227436389
train loss:2.254198345937388
train loss:2.2528988935068073
train loss:2.2667716740112795
train loss:2.275934245380075
train loss:2.291495999638741
train loss:2.257542921396383
train loss:2.258572534168787
train loss:2.2466543835208195
train 

train loss:1.5285008195896006
train loss:1.6444081117203084
train loss:1.733642162619706
train loss:1.5246690618200018
train loss:1.30242722125325
train loss:1.630312708318722
train loss:1.4183959115339753
train loss:1.432823395625245
train loss:1.5190413592798588
train loss:1.5921289959810023
train loss:1.5705748015948455
train loss:1.604194876294475
train loss:1.438748578815353
train loss:1.5595890854324592
train loss:1.5895526237136712
train loss:1.5199335911032645
train loss:1.3972557507255727
train loss:1.6095131026732516
train loss:1.5853897209847307
train loss:1.5815010344664386
train loss:1.1994584572692408
train loss:1.3834714808259454
train loss:1.499060998222238
train loss:1.451516467986579
train loss:1.698311786509855
train loss:1.490200433349479
train loss:1.4342631144927152
train loss:1.6102904869216437
train loss:1.4459333991562764
train loss:1.5430233680015661
train loss:1.5493157688748547
train loss:1.5444072422059474
train loss:1.401204098764825
train loss:1.420626191

train loss:1.3799027008700733
train loss:1.424723366954522
train loss:1.3631053249166774
train loss:1.1470443889621618
train loss:1.2996133760266309
train loss:1.5083907105573393
train loss:1.2609683793883033
train loss:1.4375647940490168
train loss:1.3435309661254358
train loss:1.2693440501445763
train loss:1.2990303686123374
train loss:1.2326364997487096
train loss:1.3135831400791755
train loss:1.2004586440456397
train loss:1.3684205005841281
train loss:1.309060800611027
train loss:1.4071892063095768
train loss:1.4677087325826954
train loss:1.2326375513858614
train loss:1.2610721429185385
train loss:1.2861327770482285
train loss:1.465528743542279
train loss:1.4882742181733744
train loss:1.3740161650866982
train loss:1.3192896538641563
train loss:1.1867041918513968
train loss:1.1906174959395017
train loss:1.3141879283463564
train loss:1.5100042721151363
train loss:1.3199523097138148
train loss:1.2099203569210846
train loss:1.4354005910874212
train loss:1.5422182978497956
train loss:1.

train loss:1.1879127401205545
train loss:1.1894763903524603
train loss:1.1614764550237764
train loss:1.0686773792916147
train loss:1.1072132410808797
train loss:1.1713442376320573
train loss:1.2111262194070351
train loss:1.2834719459511605
train loss:1.2470698162181735
train loss:1.3072688826197898
train loss:1.3876682347698792
train loss:1.1948734555141451
train loss:1.266315063506236
train loss:1.1325555892167274
train loss:1.1992676257240482
train loss:1.383820375850522
train loss:1.2258468970092362
train loss:1.172940086492634
train loss:1.4737635474179713
train loss:1.0588373759998069
train loss:1.258098905912693
train loss:1.2479278546634718
train loss:1.3328419007518741
train loss:1.3926177197475678
train loss:1.1597225705020564
train loss:1.088452441780901
train loss:1.1831571303073884
train loss:1.3121363181957892
train loss:1.2598308608643862
train loss:1.2363394683621665
train loss:1.295251384040067
train loss:1.2464945815165764
train loss:1.299536606935729
train loss:1.3778

train loss:1.1222983752273634
train loss:1.0885618555728591
train loss:1.198461845834542
train loss:1.1298160276677955
train loss:1.130980137463674
train loss:1.0734738034370148
train loss:1.238032951558474
train loss:1.1839394921924586
train loss:1.1289638828681814
train loss:1.2030521687408258
train loss:1.0509486419198821
train loss:1.2632302298867313
train loss:1.1814982020196545
train loss:1.1326292497837245
train loss:1.0030441145467968
train loss:1.114580820203353
train loss:1.235190722065218
train loss:1.1036564858106974
train loss:1.3508499271494352
train loss:1.2077683722310455
train loss:1.2015676126706039
train loss:1.044261910350494
train loss:1.2604144731060332
train loss:1.259428752454408
train loss:1.2311949230304022
train loss:1.0871857522865924
train loss:1.3796078718298377
train loss:1.0780280138628333
train loss:1.1842057724027222
train loss:1.1463908645106797
train loss:1.190306228837472
train loss:1.140837057223375
train loss:1.1758926995708463
train loss:1.322835

...

train loss:0.7879743315228103
train loss:0.9057188066851406
train loss:0.9012536612470172
train loss:0.9773321878186928
train loss:0.8271342806677581
train loss:0.8292502917730706
train loss:0.9272229440237518
train loss:0.8352232838015723
train loss:0.883087345462271
train loss:0.8085822668247544
train loss:1.0041362562631908
train loss:0.682774688580181
train loss:1.143950652041266
train loss:0.8470817588458881
train loss:1.0210627246438992
train loss:0.8518095764696858
train loss:0.8409885378307898
train loss:0.9818943825511615
train loss:0.9594594623551298
train loss:0.7542108765836708
train loss:1.0396426010084772
train loss:0.9673463919858021
train loss:0.9977996745000989
train loss:0.8763468061450952
train loss:1.0564901445693835
train loss:0.787870379139444
train loss:0.7560877595828457
train loss:0.7471210321511883
train loss:0.8928934506638929
train loss:0.6804515470242275
train loss:0.9669063926783434
train loss:0.8649222751786105
train loss:1.0557992733617767
train loss:1.0

train loss:0.8006856085627784
train loss:0.9281215024048389
train loss:0.9812214309309407
train loss:0.7164660267087848
train loss:0.9220463394553805
train loss:0.8132578101276312
train loss:0.8774203992998919
train loss:0.6729949315661596
train loss:0.990640687720292
train loss:0.9919873489221888
train loss:0.9027922903757144
train loss:0.8619788459376941
train loss:0.9594906251379995
train loss:0.8080953448261853
train loss:0.8724811430648338
train loss:1.0417684027477234
train loss:0.9794012170008604
train loss:0.8060644092216279
train loss:0.8440623756818568
train loss:0.8508234731648485
train loss:0.7978774214748148
train loss:0.6737409995720938
train loss:0.9722460193758786
train loss:1.0569378806356384
train loss:0.8384029357737799
train loss:0.7787417171640911
train loss:1.0003136644838087
train loss:0.8600887126321222
train loss:1.0434792044894305
train loss:0.8767240383269918
train loss:0.8792381997084853
train loss:0.9937789145110711
train loss:0.9771345571309316
train loss:

train loss:0.8674463347210204
train loss:0.9359732288347195
train loss:0.8674888188380383
train loss:0.8758750191409067
train loss:1.0133809644823462
train loss:0.9979669360047372
train loss:0.9650054308300758
train loss:0.8721784184226703
train loss:0.9013382156084968
train loss:0.8886693272724179
train loss:0.6724013536651259
train loss:0.8026466266346897
train loss:0.8244638167460816
train loss:1.0306933286422255
train loss:0.7416161748024757
train loss:0.9842839184179323
train loss:0.8934885122764272
train loss:0.8804402560014559
train loss:0.8130476545785537
train loss:0.8198353821583588
train loss:0.9459386227307344
train loss:0.9753739608499429
train loss:1.010078140273125
train loss:0.9424782441278129
train loss:0.8571393938380528
train loss:0.9369028368119156
train loss:1.1438134451046
train loss:0.9218423709209657
train loss:0.895207743945777
train loss:0.9868930861812467
train loss:0.8794169457519264
train loss:0.7923321063759148
train loss:1.0204981856521373
train loss:0.91

train loss:0.9994429672998435
train loss:1.0704113460924294
train loss:0.9498942705597783
train loss:1.0443532663013402
train loss:0.8045918315005387
train loss:0.9532263418577039
train loss:1.0367949745373246
train loss:0.9747955912990187
train loss:0.6588891882964671
train loss:0.8183901741462022
train loss:0.7985467583814363
train loss:0.9972798881329232
train loss:0.9034753340501845
train loss:0.915352620737475
train loss:0.9405682743477414
train loss:0.8579521721260867
train loss:0.8448001190573896
train loss:1.0961914674478952
train loss:0.7612910460441756
train loss:0.8251604409639208
train loss:0.9327885446272235
train loss:0.8408470031737678
train loss:0.9458926865054655
train loss:0.9958629172208766
train loss:1.0477655877057042
train loss:1.1259966087953164
train loss:0.9547028020423879
train loss:0.8870826847334466
train loss:1.0221667751344174
train loss:0.8998965503668884
train loss:0.8771649583496098
train loss:0.8506935938932173
train loss:0.9463308867751984
train loss:

train loss:0.8508391744914282
train loss:0.9625645279297576
train loss:0.8995111572585331
train loss:0.8961106282286768
train loss:0.9848277978916358
train loss:1.1815975025395908
train loss:0.8397410223016444
train loss:0.905524509050707
train loss:1.0136644135932817
train loss:1.0530288649285584
train loss:0.9542575543056558
train loss:0.957708112599443
train loss:0.863538557173652
train loss:0.8054893145427141
train loss:0.9344866868382938
train loss:0.9848997219195534
train loss:0.828177818347482
train loss:0.981274944048757
train loss:0.7510572028640086
train loss:1.0459680674405287
train loss:0.7308137552232743
train loss:0.8969009834658802
train loss:0.9931571741029964
train loss:0.8653752763046064
train loss:1.145807441991632
train loss:0.8960099638885994
train loss:0.8845029536708505
train loss:0.807496710547386
train loss:0.9691474379563024
train loss:1.0292626010373773
train loss:0.8162930567841247
train loss:0.8686698822664334
train loss:0.7931396582773799
train loss:0.9916

In [51]:
import pickle

In [52]:
network = DeepConvNet()
network.load_params("deep_convnet_params.pkl")

In [53]:
sampled = 10000
x_test = x_test[:sampled]
t_test = t_test[:sampled]

In [54]:
print("caluculate accuracy (float64) ... ")
print(network.accuracy(x_test, t_test))

caluculate accuracy (float64) ... 
0.9935


In [55]:
x_test = x_test.astype(np.float16)
for param in network.params.values():
    param[...] = param.astype(np.float16)

In [56]:
print("caluculate accuracy (float16) ... ")
print(network.accuracy(x_test, t_test))

caluculate accuracy (float16) ... 
0.9935
